In [2]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    device = torch.device("cuda")
else:
    print("CUDA is not available.")
    device = torch.device("cpu")

# Create a tensor
x = torch.rand(5, 3)

# Move the tensor to the GPU
x = x.to(device)

# Print the tensor
print(x)

CUDA is not available.
tensor([[0.4379, 0.4270, 0.3766],
        [0.5798, 0.0222, 0.2205],
        [0.4127, 0.3463, 0.0274],
        [0.1939, 0.0340, 0.0344],
        [0.6373, 0.2450, 0.6115]])
